In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Planning**   - 

****Please run Offline learning code first-> I am providing code below ,and you can also refer Bhavya Pranav Tandra's code ****

In [ ]:
#import time
import numpy as np
import matplotlib.pyplot as plt
import torch


#from rolf.utils import LinearDecay
#horizon_decay = LinearDecay()
batch = pretrain_buffer.sample(batch_size)
ob, ac = batch["ob"], batch["ac"]
#meta_agent = SkiMoMetaAgent(cfg, ob_space, meta_ac_space)
#skill_agent.set_step(step)



# This is for estimate_value
class Critic(nn.Module):
    def __init__(self, input_dim, hidden_dims, ensemble, activation):
        super().__init__()
        self._ensemble = ensemble
        h = hidden_dims
        assert len(h) > 0
        self.fcs = nn.ModuleList(
            [
                nn.Sequential(
                    nn.Linear(input_dim, h[0]),
                    nn.LayerNorm(h[0]),
                    nn.Tanh(),
                    MLP(h[0], 1, h[1:], activation, small_weight=True),
                )
                for _ in range(ensemble)
            ]
        )

    def forward(self, state, ac=None):
        if ac is not None:
            state = torch.cat([state, ac], dim=-1)
        q = [fc(state).squeeze(-1) for fc in self.fcs]
        return q[0] if self._ensemble == 1 else q
    
critic = Critic(50+10,[256]*4 ,1, "elu")

@torch.no_grad()  
def estimate_value(self, state, ac, horizon):
    """Imagine a trajectory for `horizon` steps, and estimate the value."""
    value, discount = 0, 1
    for t in range(horizon):
        #state, reward = self.model.imagine_step(state, ac[t])
        reward = crtic(state, ac[t])
        state = dynamics(state, ac[t])
        value += discount * reward
        discount *= 0.99  #rl_discount: 0.99
    value += discount * torch.min(*critic(state, skill_policy(state)))
    return value



def plan(ob, prev_mean=None, is_train=True):  # this method is responsible for 
        """Plan given an observation `ob`."""
        #cfg = self._cfg
        horizon = int(horizon_decay(step=0))
        #horizon_decay = LinearDecay(1, cfg.n_skill, cfg.horizon_step)

        #state = self.model.encoder(ob)
        state =state_encoder(ob)  # you need run this 
        
        # Sample policy trajectories.
        z = state.repeat(num_policy_traj, 1)
        policy_ac = []
        for t in range(horizon):
            #policy_ac.append(self.actor.act(z))
            
            policy_ac.append(skill_policy(z)) # it will call forward method
            z = dynamics(z, policy_ac[t])
        policy_ac = torch.stack(policy_ac, dim=0)

        # CEM optimization.
        z = state.repeat(num_policy_traj + num_sample_traj, 1)
        mean = torch.zeros(horizon, 10, device=device) # ac_dim =skill_dim: 10
        std = 2.0 * torch.ones(horizon, 10, device=device)
        if prev_mean is not None and horizon > 1 and prev_mean.shape[0] == horizon:
            mean[:-1] = prev_mean[1:]

        for _ in range(6):# cem_iter: 6 , we can have differnt graphs ,if you change cem_iter 
            # cem_iter 
            sample_ac = mean.unsqueeze(1) + std.unsqueeze(1) * torch.randn(
                horizon, 512, 10, device=device # ac_dim =skill_dim: 10 ,num_sample_traj: 512
            )
            sample_ac = torch.clamp(sample_ac, -0.999, 0.999)

            ac = torch.cat([sample_ac, policy_ac], dim=1)

            imagine_return = estimate_value(z, ac, horizon).squeeze(-1)
            _, idxs = imagine_return.sort(dim=0)
            idxs = idxs[-64] # num_elites: 64
            elite_value = imagine_return[idxs]
            elite_action = ac[:, idxs]

            # Weighted aggregation of elite plans.
            score = torch.exp(0.5 * (elite_value - elite_value.max())) # cem_temperature: 0.5
            score = (score / score.sum()).view(1, -1, 1)
            new_mean = (score * elite_action).sum(dim=1)
            new_std = torch.sqrt(
                torch.sum(score * (elite_action - new_mean.unsqueeze(1)) ** 2, dim=1)
            )

            mean =  0.1* mean + (1 - 0.1) * new_mean # cem_momentum: 0.1
            
            std = torch.clamp(new_std,std_decay(step=0), 2) # num_elites: 64

        # Sample action for MPC.
        score = score.squeeze().cpu().numpy()
        ac = elite_action[0, np.random.choice(np.arange(64), p=score)] #num_elites: 64
        if is_train:
            ac += std[0] * torch.randn_like(std[0])
        return torch.clamp(ac, -0.999, 0.999), mean
    
    
        

In [ ]:
# Here I need learned perameter from Downstream RL , for that i need results from my parter but he not able to provide in given so i did it ummy implementation,

# I think I can do Downstream RL then run planning 

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Dummy implementations 
def state_encoder(ob):
    return torch.randn(1, 50)

def skill_policy(z):
    return torch.randn(1, 10)

def dynamics(z, policy_ac):
    return z + policy_ac

def estimate_value(z, ac, horizon):
    return torch.randn(len(ac))

def horizon_decay(step):
    return 10  # Replace with actual implementation

def std_decay(step):
    return 1  # Replace with actual implementation

# The planning function with CEM iterations and distribution data collection
def plan(ob, prev_mean=None, is_train=True):
    horizon = horizon_decay(step=0)
    state = state_encoder(ob)
    num_policy_traj = 100  # Number of policy trajectories to sample
    num_sample_traj = 512  # Number of sample trajectories for CEM
    device = device  
    
    z = state.repeat(num_policy_traj, 1)
    policy_ac = []
    for t in range(horizon):
        policy_ac.append(skill_policy(z))
        z = dynamics(z, policy_ac[t])
    policy_ac = torch.stack(policy_ac, dim=0)
    
    z = state.repeat(num_policy_traj + num_sample_traj, 1)
    mean = torch.zeros(horizon, 10, device=device)
    std = 2.0 * torch.ones(horizon, 10, device=device)
    
    mus, sigmas, all_samples = [], [], []
    for _ in range(8):  # cem_iter: 8
        sample_ac = mean.unsqueeze(1) + std.unsqueeze(1) * torch.randn(
            horizon, num_sample_traj, 10, device=device
        )
        sample_ac = torch.clamp(sample_ac, -0.999, 0.999)
        
        ac = torch.cat([sample_ac, policy_ac], dim=1)
        imagine_return = estimate_value(z, ac, horizon).squeeze(-1)
        _, idxs = imagine_return.topk(64)  # num_elites: 64
        elite_value = imagine_return[idxs]
        elite_action = ac[:, idxs]
        
        score = F.softmax(0.5 * (elite_value - elite_value.max()), dim=0)
        new_mean = (score.unsqueeze(-1) * elite_action).sum(dim=1)
        new_std = torch.sqrt((score.unsqueeze(-1) * (elite_action - new_mean.unsqueeze(1)) ** 2).sum(dim=1))
        
        mean = 0.1 * mean + (1 - 0.1) * new_mean
        std = torch.clamp(new_std, std_decay(step=0), 2)
        
        mus.append(mean.squeeze().detach().cpu().numpy())
        sigmas.append(std.squeeze().detach().cpu().numpy())
        all_samples.append(sample_ac.squeeze().detach().cpu().numpy())

    return mus, sigmas, all_samples

# Run the plan function to get the distributions
ob = torch.randn(1, 50)  # Dummy observation
mus, sigmas, all_samples = plan(ob)

# Plotting the Gaussian distributions
for i in range(8):
    plt.figure(figsize=(10, 5))
    sample_points = all_samples[i]
    mu, sigma = mus[i], sigmas[i]
    count, bins, ignored = plt.hist(sample_points, 30, density=True, alpha=0.5, color='g')
    plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),linewidth=2, color='r')
    plt.title(f'CEM iter={i}')
    plt.xlabel('Skill Sequences')
    plt.ylabel('Probability Density')
    plt.grid(True)
    plt.show()


# **Offline learning**

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# I need use whole cell for ob -> 
from collections import defaultdict, deque
from functools import partial

import numpy as np
import tensorflow as tf
import torch.utils.data






def _convert(value, precision):
    if isinstance(value, dict):
        return {k: _convert(v, precision) for k, v in value.items()}

    value = np.array(value)
    if np.issubdtype(value.dtype, np.floating):
        dtype = {16: np.float16, 32: np.float32}[precision]
    elif np.issubdtype(value.dtype, np.signedinteger):
        dtype = {16: np.int16, 32: np.int32}[precision]
    else:
        dtype = value.dtype
    return value.astype(dtype)


class ReplayBuffer(object):
    """Replay buffer to store trainsitions in list (deque)."""

    def __init__(self, keys, buffer_size, sample_func):
        self._capacity = buffer_size
        self._sample_func = sample_func

        # create the buffer to store info
        self._keys = keys
        self.clear()

    @property
    def size(self):
        return self._capacity

    @property
    def last_saved_idx(self):
        return self._last_saved_idx

    def clear(self):
        self._idx = 0
        self._current_size = 0
        self._last_saved_idx = -1
        self._buffer = defaultdict(partial(deque, maxlen=self._capacity))

    def store_episode(self, rollout):
        """`rollout` can be any length of transitions."""
        for k in self._keys:
            self._buffer[k].append(rollout[k])

        self._idx += 1
        if self._current_size < self._capacity:
            self._current_size += 1

    def sample(self, batch_size):
        transitions = self._sample_func(self._buffer, batch_size)
        return transitions

    def state_dict(self):
        """Returns new transitions in replay buffer."""
        assert self._idx - self._last_saved_idx - 1 <= self._capacity
        state_dict = {}
        s = (self._last_saved_idx + 1) % self._capacity
        e = (self._idx - 1) % self._capacity
        for k in self._keys:
            state_dict[k] = list(self._buffer[k])
            if s < e:
                state_dict[k] = state_dict[k][s : e + 1]
            else:
                state_dict[k] = state_dict[k][s:] + state_dict[k][: e + 1]
            assert len(state_dict[k]) == self._idx - self._last_saved_idx - 1
        self._last_saved_idx = self._idx - 1
        
        return state_dict

    def append_state_dict(self, state_dict):
        """Adds transitions to replay buffer."""
        for k in self._keys:
            self._buffer[k].extend(state_dict[k])

        n = len(state_dict["ac"])
        self._last_saved_idx += n
        self._idx += n
        


class ReplayBufferEpisode(object):
    """Replay buffer storing each episode in file."""

    def __init__(self, keys, buffer_size, sample_func, precision=32):
        super().__init__()
        self._capacity = buffer_size
        self._sample_func = sample_func
        self._precision = precision

        # create the buffer to store info
        self._keys = keys
        self.clear()

    @property
    def size(self):
        return self._capacity

    @property
    def buffer(self):
        return self._buffer

    def clear(self):
        self._idx = 0
        self._new_episode = True
        self._last_saved_idx = -1
        self._buffer = deque(maxlen=self._capacity)
        self._rollout = {k: [] for k in self._keys}

    def store_episode(self, rollout, include_last_ob=True, one_more_ob=False):
        """Stores `rollout` into `self._buffer`.

        Args:
            rollout: A dictionary of lists of transitions.
            include_last_ob: Let each episode include the last observations, such that len(ob) = len(ac) + 1.
            one_more_ob: A collected rollout will contain one more observations than actions.
        """
        assert "done" in rollout
        rollout_len = len(rollout["done"])
        for i in range(rollout_len):
            if one_more_ob and include_last_ob and len(self._rollout["ob"]) == 0:
                # Add the initial observation.
                self._rollout["ob"].append(rollout["ob"].pop(i))

            for k in self._keys:
                self._rollout[k].append(rollout[k][i])

            if rollout["done"][i]:
                # Add the last observation.
                if include_last_ob:
                    if "ob_next" in rollout:
                        self._rollout["ob"].append(rollout["ob_next"][i])
                    elif not one_more_ob:
                        for k in self._keys:
                            if k != "ob":
                                self._rollout[k].pop()
                        self._rollout["done"][-1] = 1

                episode = {}
                for k in self._keys:
                    episode[k] = {}
                    if isinstance(self._rollout[k][0], dict):
                        for sub_key in self._rollout[k][0]:
                            v = np.array([t[sub_key] for t in self._rollout[k]])
                            v = _convert(v, self._precision)
                            episode[k][sub_key] = torch.as_tensor(v)
                    else:
                        v = np.array(self._rollout[k])
                        episode[k] = torch.as_tensor(_convert(v, self._precision))

                self._buffer.append(episode)
                self._idx += 1
                self._rollout = {k: [] for k in self._keys}

    def sample(self, batch_size):
        batch = self._sample_func(self._buffer, batch_size)
        return batch

    def state_dict(self):
        """Returns new transitions in replay buffer."""
        assert self._idx - self._last_saved_idx - 1 <= self._capacity
        state_dict = {}
        s = (self._last_saved_idx + 1) % self._capacity
        e = (self._idx - 1) % self._capacity
        l = list(self._buffer)
        if s <= e:
            state_dict = {"episodes": l[s : e + 1]}
        else:
            state_dict = {"episodes": l[s:] + l[: e + 1]}
        self._last_saved_idx = self._idx - 1
        
        return state_dict

    def append_state_dict(self, state_dict):
        """Adds transitions to replay buffer."""
        if state_dict is None:
            return
        self._buffer.extend(state_dict["episodes"])

        n = len(state_dict["episodes"])
        self._last_saved_idx += n
        self._idx += n
        


class RandomSampler(object):
    def __init__(self):
        pass

    def sample_func(self, dataset, batch_size):
        key_len = "ac" if "ac" in dataset else "ob"
        dataset_size = len(dataset[key_len])

        episode_idxs = np.random.randint(0, dataset_size, batch_size)
        t_samples = [
            np.random.randint(len(dataset[key_len][episode_idx]))
            for episode_idx in episode_idxs
        ]

        transitions = {}
        for key in dataset.keys():
            transitions[key] = [
                dataset[key][episode_idx][t]
                for episode_idx, t in zip(episode_idxs, t_samples)
            ]

        new_transitions = {}
        for k, v in transitions.items():
            if isinstance(v[0], dict):
                sub_keys = v[0].keys()
                new_transitions[k] = {
                    sub_key: np.stack([v_[sub_key] for v_ in v]) for sub_key in sub_keys
                }
            else:
                new_transitions[k] = np.stack(v)

        return new_transitions


class SeqSampler(object):
    def __init__(self, seq_len, sample_last_more=False):
        self._seq_len = seq_len
        self._sample_last_more = sample_last_more

    def sample_func(self, dataset, batch_size):
        data = []
        dataset_len = len(dataset)
        for _ in range(batch_size):
            while True:
                episode_idx = np.random.randint(dataset_len)
                episode = dataset[episode_idx].copy()
                episode_len = len(episode["done"])
                if episode_len >= self._seq_len:
                    break

            t = np.random.randint(episode_len - self._seq_len + 1)
            traj = [v[t : t + self._seq_len] for v in tf.nest.flatten(episode)]
            data.append(traj)

        batch = [np.stack([v[i] for v in data]) for i in range(len(traj))]
        return tf.nest.pack_sequence_as(episode, batch)

    def sample_func_tensor(self, dataset, batch_size):
        """Sample batch in tensor."""
        dataset_len = len(dataset)
        episode = dataset[0].copy()
        new_tensor = lambda v, l: torch.empty(
            (batch_size, l, *v.shape[1:]), dtype=v.dtype
        )
        batch = [new_tensor(v, self._seq_len) for v in tf.nest.flatten(episode)]

        for i in range(batch_size):
            while True:
                episode_idx = np.random.randint(dataset_len)
                episode = dataset[episode_idx].copy()
                episode_len = len(episode["done"])
                if episode_len >= self._seq_len:
                    break

            t = np.random.randint(episode_len - self._seq_len + 1)
            for j, v in enumerate(tf.nest.flatten(episode)):
                batch[j][i] = v[t : t + self._seq_len]

        # TODO: change cuda() to to(self._device)
        batch = [b.to(device) for b in batch]
        return tf.nest.pack_sequence_as(episode, batch)

    def sample_func_one_more_ob(self, dataset, batch_size):
        """Sample one more `ob` than other items."""
        # 0.18 sec (numpy array) -> 0.08 sec (empty cuda tensor) -> 0.067 sec (cuda at once)
        dataset_len = len(dataset)
        episode = dataset[0].copy()
        ob = episode.pop("ob")
        new_tensor = lambda v, l: torch.empty(
            (batch_size, l, *v.shape[1:]), dtype=v.dtype
        )
        batch_ob = [new_tensor(v, self._seq_len + 1) for v in tf.nest.flatten(ob)]
        batch_ep = [new_tensor(v, self._seq_len) for v in tf.nest.flatten(episode)]

        for i in range(batch_size):
            while True:
                episode_idx = np.random.randint(dataset_len)
                episode = dataset[episode_idx].copy()
                episode_len = len(episode["done"])
                if episode_len >= self._seq_len:
                    break

            if self._sample_last_more:
                t = np.random.randint(episode_len)
                t = np.clip(t, 0, episode_len - self._seq_len)
            else:
                t = np.random.randint(episode_len - self._seq_len + 1)
            ob = episode.pop("ob")
            for j, v in enumerate(tf.nest.flatten(ob)):
                batch_ob[j][i] = v[t : t + self._seq_len + 1]
            for j, v in enumerate(tf.nest.flatten(episode)):
                batch_ep[j][i] = v[t : t + self._seq_len]

        # TODO: change cuda() to to(self._device)
        batch_ob = [b.to(device) for b in batch_ob]
        batch_ep = [b.to(device) for b in batch_ep]

        batch_ob = tf.nest.pack_sequence_as(ob, batch_ob)
        batch_ep = tf.nest.pack_sequence_as(episode, batch_ep)
        batch_ep.update(dict(ob=batch_ob))

        return batch_ep

    def sample_func_one_more_ob_d4rl(self, dataset, batch_size):
        """Sample one more `ob` than other items.
        Sampling one more ob when data has the same number of observations and actions.
        Thus, the last action of each episode will not be used.
        """

        # 0.18 sec (numpy array) -> 0.08 sec (empty cuda tensor) -> 0.067 sec (cuda at once)
        dataset_len = len(dataset)
        episode = dataset[0].copy()
        ob = episode.pop("ob")
        new_tensor = lambda v, l: torch.empty(
            (batch_size, l, *v.shape[1:]), dtype=v.dtype
        )
        batch_ob = [new_tensor(v, self._seq_len + 1) for v in tf.nest.flatten(ob)]
        batch_ep = [new_tensor(v, self._seq_len) for v in tf.nest.flatten(episode)]

        for i in range(batch_size):
            while True:
                episode_idx = np.random.randint(dataset_len)
                episode = dataset[episode_idx].copy()
                episode_len = len(episode["done"])
                if episode_len > self._seq_len:
                    break

            t = np.random.randint(episode_len - self._seq_len)
            ob = episode.pop("ob")
            for j, v in enumerate(tf.nest.flatten(ob)):
                batch_ob[j][i] = v[t : t + self._seq_len + 1]
            for j, v in enumerate(tf.nest.flatten(episode)):
                batch_ep[j][i] = v[t : t + self._seq_len]

        # TODO: change cuda() to to(self._device)
        batch_ob = [b.to(device) for b in batch_ob]
        batch_ep = [b.to(device) for b in batch_ep]

        batch_ob = tf.nest.pack_sequence_as(ob, batch_ob)
        batch_ep = tf.nest.pack_sequence_as(episode, batch_ep)
        batch_ep.update(dict(ob=batch_ob))

        return batch_ep


class HERSampler(object):
    def __init__(self, replay_strategy, replace_future, reward_func=None):
        self.replay_strategy = replay_strategy
        if self.replay_strategy == "future":
            self.future_p = replace_future
        else:
            self.future_p = 0
        self.reward_func = reward_func

    def sample_her_transitions(self, dataset, batch_size):
        key_len = "ac" if "ac" in dataset else "ob"
        dataset_size = len(dataset[key_len])

        # select which rollouts and which timesteps to be used
        episode_idxs = np.random.randint(0, dataset_size, batch_size)
        t_samples = [
            np.random.randint(len(dataset["ac"][episode_idx]))
            for episode_idx in episode_idxs
        ]

        transitions = {}
        for key in dataset.keys():
            transitions[key] = [
                dataset[key][episode_idx][t]
                for episode_idx, t in zip(episode_idxs, t_samples)
            ]

        transitions["ob_next"] = [
            dataset["ob"][episode_idx][t + 1]
            for episode_idx, t in zip(episode_idxs, t_samples)
        ]
        transitions["r"] = np.zeros((batch_size,))

        # hindsight experience replay
        for i, (episode_idx, t) in enumerate(zip(episode_idxs, t_samples)):
            replace_goal = np.random.uniform() < self.future_p
            if replace_goal:
                future_t = np.random.randint(t + 1, len(dataset["ac"][episode_idx]) + 1)
                future_ag = dataset["ag"][episode_idx][future_t]
                if self.reward_func(dataset["ag"][episode_idx][t], future_ag, None) < 0:
                    transitions["g"][i] = future_ag
            transitions["r"][i] = self.reward_func(
                dataset["ag"][episode_idx][t + 1], transitions["g"][i], None
            )

        new_transitions = {}
        for k, v in transitions.items():
            if isinstance(v[0], dict):
                sub_keys = v[0].keys()
                new_transitions[k] = {
                    sub_key: np.stack([v_[sub_key] for v_ in v]) for sub_key in sub_keys
                }
            else:
                new_transitions[k] = np.stack(v)

        return new_transitions

In [ ]:
import pickle

# Specify the path to your pickled file
file_path = '/kaggle/input/pointmaze/maze_states_skild.pkl'

# Load the data from the pickled file
with open(file_path, 'rb') as file:
    data = pickle.load(file)

In [ ]:
data_size = len(data)

In [ ]:
ac_dim = 2
#ac_dim = skill_dim
skill_horizon = 10
state_dim = 50
skill_input_dim = skill_horizon * ac_dim + state_dim
batch_size = 256
skill_dim = 10
skill_horizon = 10
n_skill = 10
horizon = n_skill * skill_horizon
sampler = SeqSampler(horizon+1)
pretrain_buffer = ReplayBufferEpisode(["ob", "ac", "done"], None, sampler.sample_func_tensor)
pretrain_val_buffer = ReplayBufferEpisode(["ob", "ac", "done"], None, sampler.sample_func_tensor)
for i, d in enumerate(data):
    if len(d["obs"]) < len(d["dones"]):
        continue  # Skip incomplete trajectories.

    new_d = dict(ob=d["obs"], ac=d["actions"], done=d["dones"])
    new_d["done"][-1] = 1.0  # Force last step to be done.
    
    if i < data_size * 0.99: #using 0.99 percent of data for training and rest for validaation
        if i == 0:
            temp_d = new_d
        #print(f'Storing Episode: {new_d}')
        pretrain_buffer.store_episode(new_d, False)
    else:
        pretrain_val_buffer.store_episode(new_d, False)

        

In [ ]:
import torch.nn as nn

In [ ]:
import torch.nn.functional as F
class RandomShiftsAug(nn.Module):
    """Random shift image augmentation.
    Adapted from https://github.com/facebookresearch/drqv2 and https://github.com/nicklashansen/tdmpc
    """

    def __init__(self):
        super().__init__()
        # self.pad = int(cfg.img_size/21) if cfg.modality == 'pixels' else None
        self._pad = 4

    def forward(self, x):
        shape = x.shape
        x = x.reshape(-1, *shape[-3:])
        x = x.permute(0, 3, 1, 2)
        n, c, h, w = x.size()
        assert h == w
        padding = tuple([self._pad] * 4)
        x = F.pad(x, padding, "replicate")
        eps = 1.0 / (h + 2 * self._pad)
        arange = torch.linspace(
            -1.0 + eps, 1.0 - eps, h + 2 * self._pad, device=x.device, dtype=x.dtype
        )[:h]
        arange = arange.unsqueeze(0).repeat(h, 1).unsqueeze(2)
        base_grid = torch.cat([arange, arange.transpose(1, 0)], dim=2)
        base_grid = base_grid.unsqueeze(0).repeat(n, 1, 1, 1)
        shift = torch.randint(
            0, 2 * self._pad + 1, size=(n, 1, 1, 2), device=x.device, dtype=x.dtype
        )
        shift *= 2.0 / (h + 2 * self._pad)
        grid = base_grid + shift
        x = F.grid_sample(x, grid, padding_mode="zeros", align_corners=False)
        x = x.permute(0, 2, 3, 1)
        x = x.reshape(*shape)
        return x


def preprocess( ob, aug=None):
        ob = ob.copy()
        for k, v in ob.items():
            if len(v.shape) >= 4:
                ob[k] = ob[k] / 255.0 - 0.5
                if aug:
                    ob[k] = aug(ob[k])
            elif True:
                shape = ob[k].shape
                ob[k] = ob[k].view(-1, shape[-1])
                ob[k] = torch.cat([ob[k][:, :2] / 40 - 0.5, ob[k][:, 2:] / 10], -1)
                ob[k] = ob[k].view(shape)
        return ob



In [ ]:
def flip(x, l=None):
    """Flip dimensions, BxT -> TxB."""
    if isinstance(x, dict):
        return [{k: v[:, t] for k, v in x.items()} for t in range(l)]
    else:
        return x.transpose(0, 1)
    
def soft_copy_network(target, source, tau):
    """Blends `target` and `source`: `target` = `tau` * `target` + `(1-tau)` * `source`."""
    for target_param, source_param in zip(target.parameters(), source.parameters()):
        target_param.data.lerp_(source_param.data, 1 - tau)


def copy_network(target, source):
    """Copies network parameters: `target` = `source`."""
    soft_copy_network(target, source, 0)

In [ ]:
#Copied Code Cell, for TanH Dist
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions
import numpy as np


# Identity
class Identity(object):
    def __init__(self, mean):
        self.mean = mean

    def mode(self):
        return self.mean

    def sample(self):
        return self.mean.detach()

    def rsample(self):
        return self.mean


class TanhIdentity(object):
    def __init__(self, mean):
        self.mean = mean

    def mode(self):
        return torch.tanh(self.mean)

    def sample(self):
        return torch.tanh(self.mean).detach()

    def rsample(self):
        return torch.tanh(self.mean)


# Categorical
class Categorical(torch.distributions.Categorical):
    def sample(self):
        return super().sample().unsqueeze(-1)

    def log_prob(self, actions):
        return super().log_prob(actions.squeeze(-1)).unsqueeze(-1)

    def entropy(self):
        return super().entropy() * 10.0  # scailing

    def mode(self):
        return self.probs.argmax(dim=-1, keepdim=True)


# Normal
class Normal(torch.distributions.Independent):
    def __init__(self, mean, std, event_dim=0):
        super().__init__(torch.distributions.Normal(mean, std), event_dim)

    def mode(self):
        return self.mean


class AddBias(nn.Module):
    def __init__(self, bias):
        super().__init__()
        self._bias = nn.Parameter(bias.unsqueeze(1))

    def forward(self, x):
        if x.dim() == 2:
            bias = self._bias.t().view(1, -1)
        else:
            bias = self._bias.t().view(1, -1, 1, 1)
        return x + bias


class DiagGaussian(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self._device = torch.device(cfg.device)
        self.logstd = AddBias(torch.zeros(cfg.action_size, device=self._device))

    def forward(self, x):
        logstd = self.logstd(torch.zeros_like(x))
        return Normal(x, logstd.exp(), 1)


# Tanh Normal
class TanhTransform(torch.distributions.TanhTransform):
    def _inverse(self, y):
        """Clamp y for numerical stability."""
        dtype = y.dtype
        y = torch.clamp(y.float(), -0.99999997, 0.99999997)
        x = torch.atanh(y)
        return x.type(dtype)


class TanhNormal_(torch.distributions.transformed_distribution.TransformedDistribution):
    """X ~ Normal(loc, scale).
    Y ~ TanhNormal(loc, scale) = tanh(X).
    """

    def __init__(self, loc, scale):
        base_dist = torch.distributions.Normal(loc, scale)
        super().__init__(base_dist, TanhTransform())

    @property
    def mean(self):
        return self.base_dist.mean.tanh()


class TanhNormal(torch.distributions.Independent):
    def __init__(self, mean, std, event_dim=0):
        super().__init__(TanhNormal_(mean, std), event_dim)

    def mode(self):
        return self.mean

    def entropy(self):
        """No analytic form. Instead, use entropy of Normal as proxy."""
        return self.base_dist.base_dist.entropy()


class SampleDist(nn.Module):
    def __init__(self, dist, samples=100):
        super().__init__()
        self.base_dist = dist
        self._samples = samples

    def __getattr__(self, name):
        return getattr(self.base_dist, name)

    @property
    def mean(self):
        samples = self.base_dist.rsample((self._samples,))
        return torch.mean(samples, 0)

    def mode(self):
        samples = self.base_dist.rsample((self._samples,))
        log_prob = self.base_dist.log_prob(samples)
        log_prob = log_prob.sum(tuple(range(1, len(log_prob.shape))))
        return samples[torch.argmax(log_prob)]

    def entropy(self):
        samples = self.base_dist.rsample((self._samples,))
        log_prob = self.base_dist.log_prob(samples)
        return -torch.mean(log_prob, 0)

    def kl(self, scale=1.0):
        """KL Divergence between `self.base_dist` and unit gaussian."""
        samples = self.base_dist.rsample((self._samples,))
        log_prob = self.base_dist.log_prob(samples)
        mean = torch.zeros_like(self.base_dist.mean)
        std = torch.ones_like(self.base_dist.mean)
        p = Normal(mean, std, len(self.base_dist.event_shape))
        log_prob_p = p.log_prob(samples * scale)
        return log_prob.mean(0) - log_prob_p.mean(0)


def mc_kl(p, q=None, n_samples=100, scale=1.0):
    """Computes monte-carlo estimate of KL divergence."""
    if q is None or q == "tanh":
        mean = torch.zeros_like(p.mean)
        std = torch.ones_like(p.mean)
        if q is None:
            q = Normal(mean, std, len(p.event_shape))
        else:
            q = TanhNormal(mean, std, len(p.event_shape))

    samples = p.rsample((n_samples,))
    log_prob_p = p.log_prob(samples)
    log_prob_q = q.log_prob(samples * scale)
    return log_prob_p.mean(0) - log_prob_q.mean(0)


def normal_kl(p, q=None):
    """Computes KL divergence based on base normal dist."""
    if q is None or q == "tanh":
        mean = torch.zeros_like(p.mean)
        std = torch.ones_like(p.mean)
        if q is None:
            q = Normal(mean, std, len(p.event_shape))
        else:
            q = TanhNormal(mean, std, len(p.event_shape))

    p_base = torch.distributions.Independent(p.base_dist.base_dist, len(p.event_shape))
    q_base = torch.distributions.Independent(q.base_dist.base_dist, len(q.event_shape))
    return torch.distributions.kl.kl_divergence(p_base, q_base)


class MixedDistribution(nn.Module):
    def __init__(self, base_dists):
        super().__init__()
        assert isinstance(base_dists, OrderedDict)
        self.base_dists = base_dists

    def __getitem__(self, key):
        return self.base_dists[key]

    def mode(self):
        return OrderedDict([(k, dist.mode()) for k, dist in self.base_dists.items()])

    def sample(self):
        return OrderedDict([(k, dist.sample()) for k, dist in self.base_dists.items()])

    def rsample(self):
        return OrderedDict([(k, dist.rsample()) for k, dist in self.base_dists.items()])

    def log_prob(self, x):
        assert isinstance(x, dict)
        log_prob = OrderedDict(
            [(k, dist.log_prob(x[k])) for k, dist in self.base_dists.items()]
        )
        return torch.stack(list(log_prob.values()), -1).sum(-1, keepdim=True)

    def entropy(self):
        return sum([dist.entropy() for dist in self.base_dists.values()])

In [ ]:
#input dimension = horizon x action_dimension + (note, state_dim mi8 be diff than ob_dim, they mentioned state_dim to be 50 in cfg)state_dimension (action dim is 4, so if our horizon is 10 i.e 1 skill = 10 actions, then total action input len = 10x 2 = 20, and we add states to it so 10 x 2 + 50 = 70)
#skill dimension is 10, defult given in the config file 
#to the forward function concatenate actions and states, for horizon step

#In the og code they returned tanh normal distribution: return TanhNormal(mean, std, event_dim=1)
#However, they calculate behavioural_cloning_loss on after sampling from dist and kl_div directly using the dist.
class SkillEncoder(nn.Module):
    
    def __init__(self, action_obs_dimension , skill_dimension , hidden_dimension ):
        super(SkillEncoder,self).__init__()
        self.input_dim = action_obs_dimension
        self.skill_dim = skill_dimension
        self.hidden_dims = hidden_dimension
        
        #Copied Code Begins
        init_std = 5
        self._raw_init_std = np.log(np.exp(init_std) - 1)
        self._min_std = 1e-4
        self._mean_scale = 5
        #Copied Code Ends
        
        self.encoder = nn.Sequential(nn.Linear(self.input_dim,self.hidden_dims[0]),
                                     nn.ELU(),
                                     nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
                                     nn.ELU(),
                                     nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
                                     nn.ELU(),
                                     nn.Linear(self.hidden_dims[2], self.hidden_dims[3]),
                                     nn.ELU())
        
        
        self.mean = nn.Linear(self.hidden_dims[3], self.skill_dim)
        self.std = nn.Linear(self.hidden_dims[3], self.skill_dim)
        
        self.activation = nn.ELU()
        
    def forward(self,trajectory):
        
        hidden_state = self.encoder(trajectory)
        hidden_state = self.activation(hidden_state)
        mean = self.mean(hidden_state)
        std = self.std(hidden_state)
        
        #Copied Code Begins
        std = F.softplus(std + self._raw_init_std) + self._min_std
        mean = self._mean_scale * (mean / self._mean_scale).tanh()
        #Copied Code Ends
        
        #return mean, variance
        return TanhNormal(mean, std, event_dim=1)

#paper has skill policy with 2 parameters (skill and state, not just skill), so code also has input dim to be skill_dim + state_dim (encoded_state_dim )
#in code of forward as well, they concat ll_embed(encoded state) with action
class SkillPolicy(nn.Module):
    
    def __init__(self, conditioned_skill_dimension, action_dimension, hidden_dimension):
        
        super(SkillPolicy, self).__init__()
        
        #Copied Code Begins
        init_std = 5
        self._raw_init_std = np.log(np.exp(init_std) - 1)
        self._min_std = 1e-4
        self._mean_scale = 5
        #Copied Code Ends
        
        self.input_dimension = conditioned_skill_dimension
        self.output_dimension = action_dimension
        self.hidden_dims = hidden_dimension
        """self.decoder = nn.Sequential(nn.Linear(self.input_dimension,self.hidden_dims[0]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[2], self.hidden_dims[3]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[3], self.output_dimension))"""
        self.decoder = nn.Sequential(nn.Linear(self.input_dimension,self.hidden_dims[0]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[2], self.hidden_dims[3]),
                                    nn.ELU())
        self.mean = nn.Linear(self.hidden_dims[3], self.output_dimension)
        self.std = nn.Linear(self.hidden_dims[3], self.output_dimension)
        
        
    def forward(self,skill_latent, state_latent, deterministic = False, return_dist = False):
        
        conditioned_latent = torch.cat([skill_latent,state_latent], -1)
        
        
        actions = self.decoder(conditioned_latent)
        mean = self.mean(actions)
        std = self.std(actions)
        std = F.softplus(std + self._raw_init_std) + self._min_std
        mean = self._mean_scale * (mean / self._mean_scale).tanh()
        dist = TanhNormal(mean, std, event_dim=1)
        
        action = dist.mode() if deterministic else dist.rsample()
        if return_dist:
            return action, dist
        return action
        
        #return actions

class StateEncoder(nn.Module):
    
    def __init__(self, state_dimension, encoded_dimension, hidden_dimension):
        
        super(StateEncoder, self).__init__()
        self.input_dimension = state_dimension
        self.encoded_dimension = encoded_dimension
        self.hidden_dims = hidden_dimension
        self.encoder = nn.Sequential(nn.Linear(self.input_dimension,self.hidden_dims[0]),
                                    nn.ReLU(),
                                    nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
                                    nn.ReLU(),
                                    nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
                                    nn.ReLU(),
                                    nn.Linear(self.hidden_dims[2], self.hidden_dims[3]),
                                    nn.ReLU(),
                                    nn.Linear(self.hidden_dims[3], self.encoded_dimension))
        self.activation = nn.ReLU()
        
    def forward(self,state):
        
        hidden = self.encoder(state)
        out = self.activation(hidden)
        
        
        
        return out

class ObservationDecoder(nn.Module):
    
    def __init__(self, encoded_dimension, observation_dimension, hidden_dimension):
        super(ObservationDecoder, self).__init__()
        self.output_dimension = observation_dimension
        self.input_dimension = encoded_dimension
        self.hidden_dims = hidden_dimension
        self.decoder = nn.Sequential(nn.Linear(self.input_dimension,self.hidden_dims[0]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[2], self.hidden_dims[3]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[3], self.output_dimension))
        
    def forward(self, state_latent):
        
        observation = self.decoder(state_latent)
        return observation
    
class SkillPrior(nn.Module):
    
    def __init__(self, initial_state_dim, encoded_dimension, hidden_dimension):
        
        super(SkillPrior, self).__init__()
        
        init_std = 5
        self._raw_init_std = np.log(np.exp(init_std) - 1)
        self._min_std = 1e-4
        self._mean_scale = 5
        
        self.input_dim = initial_state_dim
        self.encoded_dimension = encoded_dimension
        self.hidden_dims = hidden_dimension
        self.encoder = nn.Sequential(nn.Linear(self.input_dim,self.hidden_dims[0]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
                                    nn.ELU(),
                                    nn.Linear(self.hidden_dims[2], self.hidden_dims[3]),
                                    nn.ELU())
        
        self.mean = nn.Linear(self.hidden_dims[3], self.encoded_dimension)
        self.std = nn.Linear(self.hidden_dims[3], self.encoded_dimension)
        self.activation = nn.ReLU()
        
    def forward(self,state):
        
        hidden = self.encoder(state)
        hidden = self.activation(hidden)
        mean = self.mean(hidden)
        std = self.std(hidden)
        std = F.softplus(std + self._raw_init_std) + self._min_std
        mean = self._mean_scale * (mean / self._mean_scale).tanh()
        return TanhNormal(mean, std, event_dim=1)
        
        #return mean, var
    
class SkillDynamics(nn.Module):
    
    def __init__(self, skill_and_embed_dim, embed_state_dim, hidden_dim):
        
        super(SkillDynamics, self).__init__()
        self.input_dim = skill_and_embed_dim
        self.output_dim = embed_state_dim
        self.hidden_dims = hidden_dim
        self.dynamics  = nn.Sequential(nn.Linear(self.input_dim,self.hidden_dims[0]),
                                      nn.ReLU(),
                                      nn.Linear(self.hidden_dims[0], self.hidden_dims[1]),
                                      nn.ReLU(),
                                      nn.Linear(self.hidden_dims[1], self.hidden_dims[2]),
                                      nn.ReLU(),
                                      nn.Linear(self.hidden_dims[2], self.hidden_dims[3]),
                                      nn.ReLU(),
                                      nn.Linear(self.hidden_dims[3], self.output_dim),
                                      nn.ReLU())
        
    def forward(self, embed_state, skill):
        
        inp = torch.cat([embed_state, skill], dim = -1)
        next_embed_state = self.dynamics(inp)
        
        return next_embed_state
    
class OfflineLearningModel(nn.Module):
    
    def __init__(self, trajectory_dim, skill_dim, action_dim, state_dim, embed_state_dim, hidden_dims):
        
        super(OfflineLearningModel, self).__init__()
        self.skill_encoder = SkillEncoder(trajectory_dim, skill_dim , hidden_dims)
        self.skill_policy = SkillPolicy(skill_dim, action_dim, hidden_dims)
        self.state_encoder = StateEncoder(state_dim, embed_state_dim, hidden_dims)
        self.skill_prior = SkillPrior(state_dim, embed_state_dim, hidden_dims)
        self.observation_decoder = ObservationDecoder(embed_state_dim, state_dim, hidden_dims)
        self.dynamics_model = SkillDynamics(skill_dim+embed_state_dim, embed_state_dim, hidden_dims)
        
        
    def reparameterization(self, mean, variance):
        
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * variance
        
    def forward(self,trajectories, initial_state, states, actions):
        
        skill = self.reparameterization(self.skill_encoder(trajectories))
        reconstructed_actions = self.skill_policy(skill)
        encoded_state = self.state_encoder(states[0])
        reconstructed_state = self.observation_decoder(encoded_state)
        next_embed_state = self.dynamics_model(encoded_state, skill)
        skill_prior_embed_state = self.skill_prior(initial_state)
        true_final_embed_state = self.reparameterization(self.state_encoder(states[-1]))
        
        return skill, reconstructed_actions, encoded_state, reconstructed_state, next_embed_state, skill_prior_embed_state, true_final_embed_state
        

In [ ]:
mse = nn.MSELoss(reduction="none")

state_dimension = 4
encoded_dimension = 50
hidden_dimension = [256,256,256, 256]
state_encoder = StateEncoder(state_dimension, encoded_dimension, hidden_dimension)

#  state =state_encoder(ob)


action_obs_dimension = skill_input_dim #70
skill_dimension = skill_dim #10
hidden_dimension = [256,256,256,256]
skill_encoder = SkillEncoder(action_obs_dimension , skill_dimension , hidden_dimension)

conditioned_skill_dim = state_dim + skill_dim #50+10
skill_policy = SkillPolicy(conditioned_skill_dim, ac_dim, hidden_dimension)

state_enc = StateEncoder(state_dimension, encoded_dimension, hidden_dimension)
target_state_enc = StateEncoder(state_dimension, encoded_dimension, hidden_dimension)
copy_network(target_state_enc, state_enc)

obs_decoder = ObservationDecoder(encoded_dimension, state_dimension, hidden_dimension)
dynamics = SkillDynamics(encoded_dimension+10, encoded_dimension, hidden_dimension)
skill_prior = SkillPrior(encoded_dimension,skill_dim, hidden_dimension)

hl_losses = []
ll_losses = []
hl_actor_losses = []
hl_model_losses = []
ll_actor_losses = []
ll_vae_kl_losses = []
vae_kl_divs = []
consistency_losses = []
hl_recon_losses = []
actor_stds = []
losses = []

B, H, L = batch_size, skill_horizon, n_skill
batch_size =128
def flip(x, l=None):
            """Flip dimensions, BxT -> TxB."""
            if isinstance(x, dict):
                return [{k: v[:, t] for k, v in x.items()} for t in range(l)]
            else:
                return x.transpose(0, 1)


"""
def train(self, batch, is_train=True):
    

    # ob: Bx(LxH+1)x`ob_dim`, ac: Bx(LxH+1)x`ac_dim`
    ob, ac = batch["ob"], batch["ac"]
    o = dict(ob=ob)
    o = preprocess(o, aug=self._aug)
    if ac.shape[1] == L * H + 1:
        ac = ac[:, :-1, :]

    with torch.autocast(device, enabled=True):
        # Trains skill policy and skill embedding space.
        
        encoded_state = state_encoder(o['ob'])
        
        x = ac.view(B, L, -1)
        x = torch.cat([x, encoded_state[:, :-1:H]], dim=-1)
        
        
        skill_dist = skill_encoder(x)
        sampled_skill = skill_dist.rsample()

        if True:
            # LL behavioral cloning loss.
            z_repeat = sampled_skill.unsqueeze(-2).expand(-1, -1, H, -1)
            
            
            ac_pred = skill_policy(encoded_state[:, :-1, :].view(B, L, H, -1), z_repeat, deterministic=True)
            
            
            ll_actor_loss = (
                1 * mse(ac_pred, ac.view(ac_pred.shape)).mean()
            )

            # LL embedding regularization loss.
            vae_kl_div = mc_kl(skill_dist, "tanh")
            vae_kl_div_clipped = torch.clamp(
                vae_kl_div, -100, 100
            ).mean()
            
            ll_vae_kl_loss = 1e-4 * vae_kl_div_clipped.mean()
        

        

        # Trains skill dynamics model and skill prior.
        hl_o = dict(ob=o["ob"][:, ::H])
        
        
        
        

        hl_feat = flip(state_encoder(hl_o['ob']))
        with torch.no_grad():
            hl_feat_target = flip(target_state_enc(hl_o['ob']))
        hl_ac = flip(sampled_skill)
        
        

        # HL observation reconstruction loss.
        
        
        obs_pred = obs_decoder(hl_feat)
        hl_recon_losses = mse(flip(hl_o),obs_pred).sum()
        
        h = h_next_pred = hl_feat[0]
        consistency_loss = 0
        hs = [h]
        hl_o = flip(hl_o, L + 1)
        
        
        joint_training = True

        
        for t in range(L):
            h = h_next_pred
            a = hl_ac[t] if joint_training else hl_ac[t].detach()
            h_next_pred = dynamics(h, a)
            h_next_target = hl_feat_target[t + 1]
            rho = 0.5 ** t
            consistency_loss += rho * mse(h_next_pred, h_next_target).mean(dim=1)
            hs.append(h_next_pred)
        
        
        hl_model_loss = (
            1 * hl_recon_loss
            + 2 * consistency_loss.clamp(max=1e4).mean()
        )
        hl_model_loss.register_hook(lambda grad: grad * (1 / L))
        
    
    

    # HL skill prior loss.
    if not joint_training:
        skill_prior_dist = skill_prior(hl_feat[:-1].detach())
    else:
        skill_prior_dist = skill_prior(hl_feat[:-1])
    skill_dist_detached = TanhNormal(
        flip(skill_dist.base_dist.base_dist.loc.detach()),
        flip(skill_dist.base_dist.base_dist.scale.detach()),
        1,
    )
    
    
    hl_actor_loss = 1 * mc_kl(skill_dist_detached, skill_prior_dist).mean()

    hl_loss = hl_actor_loss + hl_model_loss
    ll_loss = ll_actor_loss + ll_vae_kl_loss
    total_loss = hl_loss + ll_loss
    
    hl_losses.append(hl_loss)
    ll_losses.append(ll_loss)
    hl_actor_losses.append(hl_actor_loss)
    hl_model_losses.append(hl_model_loss)
    ll_actor_losses.append(ll_actor_loss)
    ll_vae_kl_losses.append(ll_vae_kl_loss)
    vae_kl_divs.append(vae_kl_div_clipped.mean())
    consistency_losses.append(consistency_loss.mean())
    hl_recon_losses.append(hl_recon_loss)
    actor_stds.append(skill_prior_dist.base_dist.base_dist.scale.mean())
    losses.append(total_loss)
    
    
    
    
    
    
    
    info["hl_loss"] = hl_loss.item()
    info["ll_loss"] = ll_loss.item()
    info["hl_actor_loss"] = hl_actor_loss.item()
    info["hl_model_loss"] = hl_model_loss.item()
    info["ll_actor_loss"] = ll_actor_loss.item()
    info["ll_vae_kl_loss"] = ll_vae_kl_loss.item()
    info["vae_kl_div"] = vae_kl_div_clipped.mean().item()
    info["consistency_loss"] = consistency_loss.mean().item()
    info["hl_recon_loss"] = hl_recon_loss.item()
    info["actor_std"] = skill_prior_dist.base_dist.base_dist.scale.mean().item()

    if joint_training:
        joint_grad_norm = self.joint_optim.step(total_loss)
        info["joint_grad_norm"] = joint_grad_norm.item()
    else:
        hl_model_grad_norm = self.hl_model_optim.step(hl_model_loss)
        hl_actor_grad_norm = self.hl_actor_optim.step(hl_actor_loss)
        ll_actor_grad_norm = self.ll_actor_optim.step(ll_loss)
        info["hl_model_grad_norm"] = hl_model_grad_norm.item()
        info["hl_actor_grad_norm"] = hl_actor_grad_norm.item()
        info["ll_actor_grad_norm"] = ll_actor_grad_norm.item()

    if is_train:
        self._update_iter += 1
        # Update target networks.
        if self._update_iter % cfg.target_update_freq == 0:
            soft_copy_network(
                hl_agent.model_target, hl_agent.model, cfg.target_update_tau
            )
    elif cfg.env == "maze":
        # Visualize skill trajectories.
        n_vis = cfg.pretrain.n_vis
        info["model_prediction"] = self._visualize(
            o["ob"][:n_vis].detach().cpu().numpy(),
            hl_ob_pred.mode()["ob"].transpose(0, 1)[:n_vis].detach().cpu().numpy(),
        )

        with torch.no_grad():
            # HL rollout with learned skill dynamics.
            hl_imagine_embed = torch.stack(hs, dim=0)[:, :n_vis].transpose(0, 1)
            hl_imagine = hl_agent.decoder(hl_imagine_embed).mode()["ob"]

        info["skill_rollout"] = self._visualize(
            o["ob"][:n_vis].detach().cpu().numpy(),
            hl_imagine.detach().cpu().numpy(),
        )

    return info.get_dict()"""

In [ ]:
from tqdm import tqdm
epochs = 1000000
batch_size = 256
all_parameters = list(state_encoder.parameters()) + list(skill_encoder.parameters()) + list(skill_policy.parameters()) + list(obs_decoder.parameters()) + list(dynamics.parameters()) + list(skill_prior.parameters())
learning_rate = 1e-3
weight_decay = 0
aug = RandomShiftsAug()
optimizer = torch.optim.AdamW(all_parameters, lr=learning_rate, weight_decay=weight_decay)


models = [state_encoder, skill_encoder, skill_policy,obs_decoder, dynamics, skill_prior]
for epoch in range(epochs):
    batch = pretrain_buffer.sample(batch_size)
    
    ob, ac = batch["ob"], batch["ac"]
    o = dict(ob=ob)
    o = preprocess(o, aug=aug)
    if ac.shape[1] == L * H + 1:
        ac = ac[:, :-1, :]

    with torch.autocast(device, enabled=False):
        # Trains skill policy and skill embedding space.
        
        encoded_state = state_encoder(o['ob'])
        
        x = ac.view(B, L, -1)
        x = torch.cat([x, encoded_state[:, :-1:H]], dim=-1)
        
        
        skill_dist = skill_encoder(x)
        sampled_skill = skill_dist.rsample()

        if True:
            # LL behavioral cloning loss.
            z_repeat = sampled_skill.unsqueeze(-2).expand(-1, -1, H, -1)
            
            
            ac_pred = skill_policy(encoded_state[:, :-1, :].view(B, L, H, -1), z_repeat, deterministic=True)
            
            
            ll_actor_loss = (
                1 * mse(ac_pred, ac.view(ac_pred.shape)).mean()
            )

            # LL embedding regularization loss.
            vae_kl_div = mc_kl(skill_dist, "tanh")
            vae_kl_div_clipped = torch.clamp(
                vae_kl_div, -100, 100
            ).mean()
            
            ll_vae_kl_loss = 1e-4 * vae_kl_div_clipped.mean()
        

        

        # Trains skill dynamics model and skill prior.
        hl_o = dict(ob=o["ob"][:, ::H])
        
        
        
        

        hl_feat = flip(state_encoder(hl_o['ob']))
        with torch.no_grad():
            hl_feat_target = flip(target_state_enc(hl_o['ob']))
        hl_ac = flip(sampled_skill)
        
        

        # HL observation reconstruction loss.
        
        
        obs_pred = obs_decoder(hl_feat)
        hl_recon_loss = mse(flip(hl_o['ob']),obs_pred).sum()
        
        h = h_next_pred = hl_feat[0]
        consistency_loss = 0
        hs = [h]
        hl_o = flip(hl_o, L + 1)
        
        
        joint_training = True

        
        for t in range(L):
            h = h_next_pred
            a = hl_ac[t] if joint_training else hl_ac[t].detach()
            h_next_pred = dynamics(h, a)
            h_next_target = hl_feat_target[t + 1]
            rho = 0.5 ** t
            consistency_loss += rho * mse(h_next_pred, h_next_target).mean(dim=1)
            hs.append(h_next_pred)
        
        
        hl_model_loss = (
            1 * hl_recon_loss
            + 2 * consistency_loss.clamp(max=1e4).mean()
        )
        hl_model_loss.register_hook(lambda grad: grad * (1 / L))
        
    
    

    # HL skill prior loss.
    if not joint_training:
        skill_prior_dist = skill_prior(hl_feat[:-1].detach())
    else:
        skill_prior_dist = skill_prior(hl_feat[:-1])
    skill_dist_detached = TanhNormal(
        flip(skill_dist.base_dist.base_dist.loc.detach()),
        flip(skill_dist.base_dist.base_dist.scale.detach()),
        1,
    )
    
    
    hl_actor_loss = 1 * mc_kl(skill_dist_detached, skill_prior_dist).mean()

    hl_loss = hl_actor_loss + hl_model_loss
    ll_loss = ll_actor_loss + ll_vae_kl_loss
    total_loss = hl_loss + ll_loss
    
    hl_losses.append(hl_loss)
    ll_losses.append(ll_loss)
    hl_actor_losses.append(hl_actor_loss)
    hl_model_losses.append(hl_model_loss)
    ll_actor_losses.append(ll_actor_loss)
    ll_vae_kl_losses.append(ll_vae_kl_loss)
    vae_kl_divs.append(vae_kl_div_clipped.mean())
    consistency_losses.append(consistency_loss.mean())
    hl_recon_losses.append(hl_recon_loss)
    actor_stds.append(skill_prior_dist.base_dist.base_dist.scale.mean())
    losses.append(total_loss)
    
    
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    print(f'Epoch: {epoch+1},  Loss:{total_loss}')
    
    
    